<a href="https://colab.research.google.com/github/francejules22/Type2DiabetesPredictionsModel/blob/main/TypeIIDiabetesFinalGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Import Dependencies and Libraries

In [94]:
%matplotlib inline 
import numpy as np    # linear algebra 
import pandas as pd    # Data processing, Input & Output load    

import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier    # GBM algorithm


from xgboost.sklearn import XGBClassifier    # Extreme Gradient Boosting

import joblib  #Joblib is a set of tools to provide lightweight pipelining in Python (Avoid computing twice the same thing)
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score, GridSearchCV
                                    # GridSearchCV - Implements a “fit” and a “score” method
                                    # train_test_split - Split arrays or matrices into random train and test subsets
                                    # cross_val_score - Evaluate a score by cross-validation 


from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score, precision_score, accuracy_score, roc_auc_score, recall_score, roc_curve
from sklearn.metrics import make_scorer, confusion_matrix, classification_report   # Differnt metrics to evaluate the model
import pandas_profiling as pp    # simple and fast exploratory data analysis of a Pandas Dataframe

import warnings    # To avoid warning messages in the code run
warnings.filterwarnings('ignore')

## Step 2: Import Dataset

In [95]:
import pandas as pd 
diabetes = pd.read_csv('TypeIIDiabetes.csv')    
diabetes

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


## Step 3: Applying the Label Encode

In [96]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in diabetes.columns[1:] :
    diabetes[i] = le.fit_transform(diabetes[i])

diabetes

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,0,1,1,1,0,1,0,0,1,0,1,1,0,0,0,1
516,48,0,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1
517,58,0,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1
518,32,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0


## Step 4: Spliting Independent and dependent variables

a.) Independent Variables

In [97]:
x=diabetes.iloc[:,:-1].values
x

array([[40,  1,  0, ...,  1,  1,  1],
       [58,  1,  0, ...,  0,  1,  0],
       [41,  1,  1, ...,  1,  1,  0],
       ...,
       [58,  0,  1, ...,  1,  0,  1],
       [32,  0,  0, ...,  0,  1,  0],
       [42,  1,  0, ...,  0,  0,  0]])

b.) Dependent Variables

In [98]:
y=diabetes.iloc[:,-1].values
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

## Step 5: Transform Data using Standard Scaler

In [99]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x=sc.fit_transform(x)
x

array([[-0.6613669 ,  0.76509206, -0.99233705, ...,  1.29099445,
         1.38022749,  2.21564684],
       [ 0.82136224,  0.76509206, -0.99233705, ..., -0.77459667,
         1.38022749, -0.45133547],
       [-0.57899306,  0.76509206,  1.00772212, ...,  1.29099445,
         1.38022749, -0.45133547],
       ...,
       [ 0.82136224, -1.30703226,  1.00772212, ...,  1.29099445,
        -0.72451824,  2.21564684],
       [-1.32035762, -1.30703226, -0.99233705, ..., -0.77459667,
         1.38022749, -0.45133547],
       [-0.49661921,  0.76509206, -0.99233705, ..., -0.77459667,
        -0.72451824, -0.45133547]])

## Step 6: Save the transform data using joblib

In [100]:
import joblib  
joblib.dump(sc,'diabetes_transform')

['diabetes_transform']

## Step 7: Splitting the dataset into 80% training data and 20% testing data

In [101]:
X_train, X_test, y_train, y_test = tts(x, y, test_size = 0.3, random_state = 100) 

print('Train Shape: ', X_train.shape)
print('Test Shape: ', X_test.shape)

Train Shape:  (364, 16)
Test Shape:  (156, 16)


## Step 8: Construct Model using Gradient Boosting 

In [102]:
model_parameters = {'n_estimators': [10, 50, 100, 200, 500, 750, 1000], 'max_depth': [3, 5, 10],
                    'min_samples_leaf': [np.random.randint(1,10)], 'max_features': [None, 'sqrt', 'log2']}

In [103]:
model = GradientBoostingClassifier(random_state = 10)
gscv_GBM = GridSearchCV(estimator = model, 
                        param_grid = model_parameters, 
                        cv = 5, 
                        verbose = 1, 
                        n_jobs = -1,
                        scoring = 'roc_auc')

gscv_GBM.fit(X_train, y_train)

Fitting 5 folds for each of 63 candidates, totalling 315 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=10),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5, 10],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [5],
                         'n_estimators': [10, 50, 100, 200, 500, 750, 1000]},
             scoring='roc_auc', verbose=1)

### a.) Refitting the data

In [104]:
model = GradientBoostingClassifier(**gscv_GBM.best_params_)
model.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=10, max_features='sqrt',
                           min_samples_leaf=5, n_estimators=200)

### b.) Displaying model prediction and classification report

In [105]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [106]:
print('Classification report for train data is : \n',
      classification_report(y_train, train_pred))
print('Classification report for test data is : \n',
      classification_report(y_test, test_pred))

Classification report for train data is : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       148
           1       1.00      1.00      1.00       216

    accuracy                           1.00       364
   macro avg       1.00      1.00      1.00       364
weighted avg       1.00      1.00      1.00       364

Classification report for test data is : 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97        52
           1       0.99      0.98      0.99       104

    accuracy                           0.98       156
   macro avg       0.98      0.98      0.98       156
weighted avg       0.98      0.98      0.98       156



### c.) Making predictions for test data by testing the model

In [107]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
predictions

[1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0]

### d.) Checking the prediction accuracy for test data of the model

In [108]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 98.08%


### e.) Model Evaluation Metrics

In [109]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test, y_pred)))
print('Precision Score : ' + str(precision_score(y_test, y_pred)))
print('Recall Score : ' + str(recall_score(y_test, y_pred)))
print('F1 Score : ' + str(f1_score(y_test, y_pred)))

Accuracy Score : 0.9807692307692307
Precision Score : 0.9902912621359223
Recall Score : 0.9807692307692307
F1 Score : 0.9855072463768114


### f.) Confusion Matrix

In [110]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test, y_pred)))

Confusion Matrix : 
[[ 51   1]
 [  2 102]]


### g.) Saving the Gradient Boosting Model

In [111]:
joblib.dump(model, 'diabetes_model')

['diabetes_model']

### Step 9: Testing if it really works 

In [112]:
print(model.predict([[40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1]]))

[1]


In [113]:
print(model.predict([[72,1,1,0,0,0,1,0,1,1,0,1,1,1,1,0]]))

[0]


In [114]:
print(model.predict([[90,0,0,1,1,0,0,1,1,1,0,0,0,1,1,0]]))

[1]


In [115]:
print(model.predict([[58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0]]))

[1]


In [116]:
print(model.predict([[41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0]]))

[1]


In [117]:
print(model.predict([[61,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1]]))

[1]


In [118]:
print(model.predict([[21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]))

[0]


In [119]:
print(model.predict([[21,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]]))

[1]


In [120]:
print(model.predict([[21,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0]]))

[0]


In [122]:
print(model.predict([[21,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0]]))

[0]


In [123]:
print(model.predict([[21,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0]]))

[0]


In [124]:
print(model.predict([[21,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0]]))

[0]


In [127]:
print(model.predict([[90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]))

[0]


In [128]:
print(model.predict([[38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]))

[0]


In [129]:
print(model.predict([[21,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]]))

[1]


In [130]:
print(model.predict([[43,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0]]))

[0]
